## CL data analysis

Notebook for examining data from CL acquisition, including plotting, PCA denoising, NMF decomposition, max intensity wavelength mapping, and intensity distributions for different spectral ranges. Functions used here are from *utilities_CL_master.py*. 

In [2]:
%matplotlib qt

from utilities_CL_master import *
import hyperspy.api as hs
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = 'serif'
import autoreload
%load_ext autoreload


### Loading and plotting signal

Load *.hdf5*-files for the CL signal and the preview SEM image. These are converted from the *.h5* Odemis file using the *Odemis_to_hyperspy.ipynb*-Notebook.

In [3]:
# Path to signal file
path = '20-04-28 FIB/'
title =  'r1'

# Load hdf5-file, plot the interactive signal, and the total sum spectrum
s = hs.load(path + 'hdf5/' + title + '.hdf5') 
s.plot()
s.sum(axis=(0,1)).plot()

### Crop, decompose, and get the denoised data

In [4]:
s_crop = s.isig[240.:380.].deepcopy()

s_crop.decomposition(normalize_poissonian_noise = True, algorithm='RPCA_GoDec', output_dimension = 25) 
s_crop.plot_explained_variance_ratio()  # Plot scree plot
s_crop.plot_decomposition_results() # Plot eigenvalues and eigenvectors

In [5]:
s_pca = s_crop.get_decomposition_model(components = 5).deepcopy()

### NMF decomposition and plotting

In [6]:
s_nmf = s_pca.deepcopy()
s_nmf.decomposition(algorithm='nmf', output_dimension = 6)
s_nmf.plot_decomposition_results()

In [7]:
%autoreload

# Select which factors to plot (default = first three)
c = [1,3,4]

plot_nmf_loadings(signal = s_nmf, selected = c, save = path + 'nmf/' + title)
plot_nmf_factors(signal = s_nmf, selected = c, offset = 20, save = path + 'nmf/' + title)

### Plot max wavelength distribution and intensity distribution of different spectral ranges

In [14]:
%autoreload

# Plot the maps of the max intensity wavelength for each pixel
wavelength_distribution(s_pca.isig[357.:364.], cutoff = 340, save = path + 'maps/' + title + '_357-364')
wavelength_distribution(s_pca.isig[275.:290.], cutoff = 350, save = path + 'maps/' + title + '_275-290')
wavelength_distribution(s_pca.isig[325.:330.], cutoff = 350, save = path + 'maps/' + title + '_325-330')


In [58]:
%autoreload

# Plot the intensity of emission for different spectral ranges
limits = [200., 250., 300., 380., 650.]

for i in range(len(limits)-1):
    cropped = s.isig[limits[i]:limits[i+1]]
        
    intensity_map(cropped, save = path + 'intensities/' + title + '_' + str(limits[i]) + '-' + str(limits[i+1]))

### Gaussian fitting 

Function which fits *n* (max 5) Gaussians to the sum spectrum, and plots the result. 

In [51]:
%autoreload

# Crop signal to the desired range
sig = s.isig[200.:620.]

# The bounds for each paramter can either be entered manually, or the automatic ones can be used
bounds = ([0., 200., 0., 0., 200., 0., 0., 380., 0.],#, 0., 380., 0.], 
         [1., 380., 100., 1., 380., 100., 1., 650., 100.])#, 1., 650., 100.])

gaussian_fit(sig, n = 3, save = path + 'gaussian/' + title + '_full', bounds = bounds)

Gaussian 1: mean = 282.58, STD = 10.35
Gaussian 2: mean = 328.34, STD = 15.14
Gaussian 3: mean = 500.93, STD = 82.39
